## Setup

In [2]:
# Import packages
import pandas as pd
from ds_common_utils.aux.io.snowflake_tools import SnowflakeTools
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.express as px

In [3]:
# Setup connection
con = SnowflakeTools(
    role="INSIGHT_ANALYST_MERCH_DE_GENERAL_PRD",
    warehouse="INSIGHT_ANALYST_WH",
    database="BDWPRD_DE",
    schema="IA_MERCH_DE"
)

In [4]:
# Date range
start_date = "'2024-06-01'"
end_date = "'2025-05-31'"

## Data

In [5]:
# Set item range table name
table_GM = 'GM_item_range' + datetime.today().strftime('%Y_%m_%d')

In [6]:
# Create temp item table
con.execute_statement_from_sql_file(
    file='sql/0-item-range.sql',
    formatting={'table_name': table_GM,
                'filter' : "AND item_sub_department_name = '400 GROWING MEDIA' "})

snowflake_tools - 2025-07-11 10:36:22.278896+10:00 - Generating new token with 'INSIGHT_ANALYST_MERCH_DE_GENERAL_PRD' role and 'SESSION:ROLE-ANY' scope...
snowflake_tools - 2025-07-11 10:36:28.863018+10:00 - Saved token with '2025-07-11 14:36:28+10:00' expiry...


## Demographic Segments (AU)

### Overall index

In [7]:
ds = con.read_sql_file_into_pandas(
    file="sql/demographic-segments-overall.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'filter' : "item_sub_department_name = '400 GROWING MEDIA'"
    }
)

In [8]:
print(f"Total sales: ${ds['RANGE_SALES'].sum():,.2f}")
print(f"Total quantity: {int(ds['RANGE_QUANTITY'].sum()):,}")
print(f"Total customers: {int(ds['RANGE_CUSTOMERS'].sum()):,}")

Total sales: $160,691,813.73
Total quantity: 18,538,496
Total customers: 2,592,007


In [9]:
ds[["DEMOGRAPHIC_SEGMENT", "RANGE_SALES", "RANGE_SALES_SHARE", "SALES_INDEX"]]

,DEMOGRAPHIC_SEGMENT,RANGE_SALES,RANGE_SALES_SHARE,SALES_INDEX
0,Homeowner older kids,12658140.01,0.078773,0.83
1,Homeowner younger kids,21650028.46,0.134730,0.72
2,Older homeowner,87747198.46,0.546059,1.31
3,Renter no kids,9191033.47,0.057197,0.65
4,Renter with kids,5073012.15,0.031570,0.52
5,Unclassifiable,1434624.35,0.008928,1.31
6,Younger homeowner,22937776.83,0.142744,0.99


$161M in sales, 54% comes from Older homeowner
- Older homeowner over indexed - loves gardening, have large garden
- Homwowner younger kids and renters under indexed - no garden?

### Class index

In [10]:
GM_ds = con.read_sql_file_into_pandas(
    file="sql/1-demographic-segments.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'table_name': table_GM,
        'level' : 'item_class_name',
        'country' : 'AU' 
    }
)

In [11]:
summary = (
    GM_ds[["ITEM_CLASS_NAME", "TOTAL_SALES", "TOTAL_QUANTITY", "TOTAL_CUSTOMERS"]]
    .drop_duplicates(subset="ITEM_CLASS_NAME")
    .rename(columns={
        "TOTAL_SALES": "CLASS_SALES",
        "TOTAL_QUANTITY": "CLASS_QUANTITY",
        "TOTAL_CUSTOMERS": "CLASS_CUSTOMERS"
    })
)

# Compute and print total shares
total_sales = summary["CLASS_SALES"].sum()
total_quantity = summary["CLASS_QUANTITY"].sum()
total_customers = summary["CLASS_CUSTOMERS"].sum()

# Add share columns
summary["CLASS_SALES_SHARE"] = summary["CLASS_SALES"] / total_sales
summary["CLASS_QUANTITY_SHARE"] = summary["CLASS_QUANTITY"] / total_quantity
summary["CLASS_CUSTOMERS_SHARE"] = summary["CLASS_CUSTOMERS"] / total_customers

summary

,ITEM_CLASS_NAME,CLASS_SALES,CLASS_QUANTITY,CLASS_CUSTOMERS,CLASS_SALES_SHARE,CLASS_QUANTITY_SHARE,CLASS_CUSTOMERS_SHARE
0,500 COIR MULCH BLOCK HYDRO,1155603.63,60511.0,30355,0.007191,0.003264,0.006839
7,500 COIR POTTING MIX BLOCK,708514.12,84519.0,49384,0.004409,0.004559,0.011126
14,500 COMPOSTS SOIL CONDITIONERS,29642027.43,4166671.0,881094,0.184465,0.224758,0.198498
21,500 MANURE,5803884.42,1101361.0,293716,0.036118,0.059409,0.066170
28,500 MULCHES,39012025.94,3291401.0,742062,0.242775,0.177544,0.167176
35,500 POTTING MIXES,76869003.63,8938231.0,1948306,0.478363,0.482144,0.438927
42,500 SMALL BAG POTTING MIX,7500754.56,895802.0,493880,0.046678,0.048321,0.111264


In [12]:
print(f"Total sales: ${summary['CLASS_SALES'].sum():,.2f}")
print(f"Total quantity: {int(summary['CLASS_QUANTITY'].sum()):,}")
print(f"Total customers: {int(summary['CLASS_CUSTOMERS'].sum()):,}")

Total sales: $160,691,813.73
Total quantity: 18,538,496
Total customers: 4,438,797


In [13]:
GM_ds[['ITEM_CLASS_NAME', 'DEMOGRAPHIC_SEGMENT', 'SEGMENT_SALES', 'SEGMENT_SALES_SHARE', 'SALES_INDEX']]

,ITEM_CLASS_NAME,DEMOGRAPHIC_SEGMENT,SEGMENT_SALES,SEGMENT_SALES_SHARE,SALES_INDEX
0,500 COIR MULCH BLOCK HYDRO,Homeowner older kids,100865.04,0.087283,1.108040
1,500 COIR MULCH BLOCK HYDRO,Homeowner younger kids,180888.60,0.156532,1.161817
2,500 COIR MULCH BLOCK HYDRO,Older homeowner,543161.02,0.470024,0.860756
3,500 COIR MULCH BLOCK HYDRO,Renter no kids,84448.50,0.073077,1.277652
4,500 COIR MULCH BLOCK HYDRO,Renter with kids,42079.39,0.036413,1.153422
5,500 COIR MULCH BLOCK HYDRO,Unclassifiable,8000.41,0.006923,0.775459
6,500 COIR MULCH BLOCK HYDRO,Younger homeowner,196160.67,0.169747,1.189174
7,500 COIR POTTING MIX BLOCK,Homeowner older kids,55588.85,0.078458,0.996008
8,500 COIR POTTING MIX BLOCK,Homeowner younger kids,97895.37,0.138170,1.025531
9,500 COIR POTTING MIX BLOCK,Older homeowner,343730.03,0.485142,0.888443


48% sales comes from 500 POTTING MIXES, 24% from 500 MULCHES, 18% from 500 COMPOSTS SOIL CONDITIONERS, the rest 10% are minimal...	
- Renter no kids under indexed in 500 MANURE, 500 MULCHES
- Renters over indexed in 500 COIR MULCH BLOCK HYDRO, 500 COIR POTTING MIX BLOCK, 500 SMALL BAG POTTING MIX
- Homeowner younger kids over indexed in 500 COMPOSTS SOIL CONDITIONERS

## Non-Commercial (NZ)

### Class index

In [14]:
non_commercial = con.read_sql_file_into_pandas(
    file="sql/non-commercial-overall.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'table_name': table_GM,
        'level' : 'item_class_name',
        'country' : 'NZ'
    }
)

In [15]:
non_commercial

,ITEM_CLASS_NAME,TOTAL_SALES,TOTAL_QUANTITY,SALES_SHARE,QUANTITY_SHARE
0,500 COIR POTTING MIX BLOCK,144226.55,22615.0,0.007431,0.010363
1,500 COMPOSTS SOIL CONDITIONERS,5475485.76,754963.0,0.282110,0.345963
2,500 MULCHES,3082274.46,304398.0,0.158806,0.139491
3,500 POTTING MIXES,9281133.31,937747.0,0.478186,0.429725
4,500 SMALL BAG POTTING MIX,1425938.11,162482.0,0.073468,0.074458


In [16]:
print(f"Total sales: ${non_commercial['TOTAL_SALES'].sum():,.2f}")
print(f"Total quantity: {int(non_commercial['TOTAL_QUANTITY'].sum()):,}")

Total sales: $19,409,058.19
Total quantity: 2,182,205


## Commercial Industry Segment (NZ)

### Overall index

In [17]:
cis = con.read_sql_file_into_pandas(
    file="sql/commercial-industry-segments-overall.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'filter' : "item_sub_department_name = '400 GROWING MEDIA'",
        'country' : 'NZ'
    }
)

In [18]:
print(f"Total sales: ${cis['RANGE_SALES'].sum():,.2f}")
print(f"Total quantity: {int(cis['RANGE_QUANTITY'].sum()):,}")
print(f"Total customers: {int(cis['RANGE_CUSTOMERS'].sum()):,}")

Total sales: $4,075,711.62
Total quantity: 474,806
Total customers: 34,878


In [19]:
n = 8
top_n = cis.nlargest(n, "RANGE_SALES").copy()
top_n[["INDUSTRY_SEGMENT", "RANGE_SALES", "RANGE_SALES_SHARE", "SALES_INDEX"]]

,INDUSTRY_SEGMENT,RANGE_SALES,RANGE_SALES_SHARE,SALES_INDEX
0,Professional Computer and Scientific Services,672372.09,0.164970,2.00
1,Personal and Other Services,574593.10,0.140980,2.33
2,Rental Hiring and Real Estate Services,329816.04,0.080922,0.93
3,Retail and Wholesale Trade,309192.48,0.075862,1.49
4,Residential Builder,308458.70,0.075682,0.30
5,Manufacturing,294825.51,0.072337,1.40
6,Site Preparation and Development,214062.55,0.052522,0.74
7,Landscaper and Gardening Services,204666.26,0.050216,1.49


In [20]:
print(f"Top {n} segments - sales share: {top_n['RANGE_SALES_SHARE'].sum():.2%}")
print(f"Top {n} segments - quantity share: {top_n['RANGE_QUANTITY_SHARE'].sum():.2%}")
print(f"Top {n} segments - customer share: {top_n['RANGE_CUSTOMER_SHARE'].sum():.2%}")
print(f"Top {n} segments - size share: {top_n['SEGMENT_SIZE_SHARE'].sum():.2%}")

Top 8 segments - sales share: 71.35%
Top 8 segments - quantity share: 71.31%
Top 8 segments - customer share: 64.60%
Top 8 segments - size share: 45.91%


### Class index

In [21]:
# Get data
GM_is = con.read_sql_file_into_pandas(
    file="sql/2-commercial-industry-segment.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'n_commercial_segments': n, 
        'table_name': table_GM,
        'level' : 'item_class_name',
        'country' : 'NZ' 
    }
)

In [22]:
summary = (
    GM_is[["ITEM_CLASS_NAME", "TOTAL_SALES", "TOTAL_QUANTITY", "TOTAL_CUSTOMERS"]]
    .drop_duplicates(subset="ITEM_CLASS_NAME")
    .rename(columns={
        "TOTAL_SALES": "CLASS_SALES",
        "TOTAL_QUANTITY": "CLASS_QUANTITY",
        "TOTAL_CUSTOMERS": "CLASS_CUSTOMERS"
    })
)

# Compute and print total shares
total_sales = summary["CLASS_SALES"].sum()
total_quantity = summary["CLASS_QUANTITY"].sum()
total_customers = summary["CLASS_CUSTOMERS"].sum()

# Add share columns
summary["CLASS_SALES_SHARE"] = summary["CLASS_SALES"] / total_sales
summary["CLASS_QUANTITY_SHARE"] = summary["CLASS_QUANTITY"] / total_quantity
summary["CLASS_CUSTOMERS_SHARE"] = summary["CLASS_CUSTOMERS"] / total_customers

summary

,ITEM_CLASS_NAME,CLASS_SALES,CLASS_QUANTITY,CLASS_CUSTOMERS,CLASS_SALES_SHARE,CLASS_QUANTITY_SHARE,CLASS_CUSTOMERS_SHARE
0,500 COIR POTTING MIX BLOCK,13135.64,2191.0,400,0.004754,0.006801,0.011226
8,500 COMPOSTS SOIL CONDITIONERS,832162.15,118694.0,11139,0.301142,0.368409,0.312630
16,500 MULCHES,445372.80,47246.0,4491,0.161171,0.146645,0.126045
24,500 POTTING MIXES,1345234.29,140071.0,15316,0.486812,0.434760,0.429862
32,500 SMALL BAG POTTING MIX,127450.02,13978.0,4284,0.046121,0.043386,0.120236


In [23]:
print(f"Total sales: ${summary['CLASS_SALES'].sum():,.2f}")
print(f"Total quantity: {int(summary['CLASS_QUANTITY'].sum()):,}")
print(f"Total customers: {int(summary['CLASS_CUSTOMERS'].sum()):,}")

Total sales: $2,763,354.90
Total quantity: 322,180
Total customers: 35,630


In [24]:
GM_is[['ITEM_CLASS_NAME', 'INDUSTRY_SEGMENT', 'SEGMENT_SALES', 'SEGMENT_SALES_SHARE', 'SALES_INDEX']]

,ITEM_CLASS_NAME,INDUSTRY_SEGMENT,SEGMENT_SALES,SEGMENT_SALES_SHARE,SALES_INDEX
0,500 COIR POTTING MIX BLOCK,Commercial Builder,117.44,0.008941,0.411530
1,500 COIR POTTING MIX BLOCK,Manufacturing,1852.68,0.141042,1.321968
2,500 COIR POTTING MIX BLOCK,Personal and Other Services,2756.83,0.209874,1.009334
3,500 COIR POTTING MIX BLOCK,Professional Computer and Scientific Services,2996.74,0.228138,0.937616
4,500 COIR POTTING MIX BLOCK,Rental Hiring and Real Estate Services,1265.87,0.096369,0.807426
5,500 COIR POTTING MIX BLOCK,Residential Builder,1999.29,0.152203,1.363528
6,500 COIR POTTING MIX BLOCK,Retail and Wholesale Trade,1655.48,0.126030,1.126368
7,500 COIR POTTING MIX BLOCK,Site Preparation and Development,491.31,0.037403,0.482837
8,500 COMPOSTS SOIL CONDITIONERS,Commercial Builder,19449.50,0.023372,1.075813
9,500 COMPOSTS SOIL CONDITIONERS,Manufacturing,84287.44,0.101287,0.949350


## Drop item range table

In [25]:
# # Drop item range table
# con.execute_statement_from_sql_string(
#     statement='DROP TABLE bdwprd_de.ia_merch_de.{table_name};',
#     formatting={'table_name': table_GM}
#     )

## Potting Mixes Volume

### AU

In [67]:
pm_au = con.read_sql_file_into_pandas(
    file="sql/potting_mixes_volume.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'country' : 'AU'
    }
)

In [68]:
# Recalculate brand + package-level summary
brand_pkg_summary = (
    pm_au.groupby(["BRAND_CODE", "PACKAGE_LITRE"])
    .agg(
        total_sales=("TOTAL_SALES", "sum"),
        avg_volume_per_trx=("ESTIMATED_LITRES_PER_TRX", "mean"),
        avg_qty_per_trx=("AVG_QTY_PER_TRX", "mean"),
        total_transactions=("TOTAL_TRX", "sum"),
        avg_price_per_litre=("AVG_PRICE_PER_LITRE", "mean")
    )
    .reset_index()
)

brand_pkg_summary = brand_pkg_summary[brand_pkg_summary["total_sales"] > 0]

In [69]:
package_order = sorted(brand_pkg_summary["PACKAGE_LITRE"].unique())

brand_order = (
    brand_pkg_summary.groupby("BRAND_CODE")["total_sales"]
    .sum()
    .sort_values(ascending=False)
    .index.tolist()
)

fig = px.scatter(
    brand_pkg_summary,
    x="PACKAGE_LITRE",
    y="BRAND_CODE",
    size="total_sales",
    color="avg_volume_per_trx",
    color_continuous_scale="RdYlGn",
    category_orders={
        "PACKAGE_LITRE": package_order,
        "BRAND_CODE": brand_order
    },
    hover_data={
        "total_sales": ":,.0f",
        "avg_price_per_litre": ":.2f",
        "total_transactions": ":,",
        "avg_qty_per_trx": ":.2f",
        "avg_volume_per_trx": ":.2f",
        "PACKAGE_LITRE": False,
        "BRAND_CODE": False
    },
        labels={
        "total_sales": "Total Sales ($) ",
        "avg_price_per_litre": "Avg $/L ",
        "total_transactions": "Total Trx ",
        "avg_qty_per_trx": "Avg Units/Trx ",
        "avg_volume_per_trx": "Avg Volume/Trx (L) "
    },
    title="AU Potting Mixes: Sales Volume by Brand & Package Size"
)

fig.update_layout(
    width=800,
    height=800,
    template="plotly_white",
    font=dict(family="Arial", size=14, color="#333"),
    title_font=dict(size=20),
    xaxis=dict(
        tickmode="array",
        tickvals=package_order,
        ticktext=[f"{x}L" for x in package_order],
        title="",
        tickfont=dict(size=13),
        titlefont=dict(size=16)
    ),
    yaxis=dict(
        title="",
        categoryorder="array",
        categoryarray=brand_order,
        tickfont=dict(size=13),
        titlefont=dict(size=16)
    ),
    coloraxis_colorbar=dict(
        title="Avg Volume/Trx (L)",
        titlefont=dict(size=14),
        tickfont=dict(size=12)
    )
)

# Optional: double bubble size (by halving sizeref)
max_sales = brand_pkg_summary["total_sales"].max()
desired_size_max = 30  # same unit as original size_max

fig.update_traces(
    marker=dict(
        sizemode="area",
        sizeref=0.25 * max_sales / desired_size_max**2,
        sizemin=5
    )
)

fig.show()

In [70]:
fig.write_html("potting_mix_volume_by_brand_AU.html")

### NZ

In [71]:
pm_nz = con.read_sql_file_into_pandas(
    file="sql/potting_mixes_volume.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'country' : 'NZ'
    }
)

In [72]:
# Recalculate brand + package-level summary
brand_pkg_summary = (
    pm_nz.groupby(["BRAND_CODE", "PACKAGE_LITRE"])
    .agg(
        total_sales=("TOTAL_SALES", "sum"),
        avg_volume_per_trx=("ESTIMATED_LITRES_PER_TRX", "mean"),
        avg_qty_per_trx=("AVG_QTY_PER_TRX", "mean"),
        total_transactions=("TOTAL_TRX", "sum"),
        avg_price_per_litre=("AVG_PRICE_PER_LITRE", "mean")
    )
    .reset_index()
)

brand_pkg_summary = brand_pkg_summary[brand_pkg_summary["total_sales"] > 0]

In [73]:
package_order = sorted(brand_pkg_summary["PACKAGE_LITRE"].unique())

brand_order = (
    brand_pkg_summary.groupby("BRAND_CODE")["total_sales"]
    .sum()
    .sort_values(ascending=False)
    .index.tolist()
)

fig = px.scatter(
    brand_pkg_summary,
    x="PACKAGE_LITRE",
    y="BRAND_CODE",
    size="total_sales",
    color="avg_volume_per_trx",
    color_continuous_scale="RdYlGn",
    category_orders={
        "PACKAGE_LITRE": package_order,
        "BRAND_CODE": brand_order
    },
    hover_data={
        "total_sales": ":,.0f",
        "avg_price_per_litre": ":.2f",
        "total_transactions": ":,",
        "avg_qty_per_trx": ":.2f",
        "avg_volume_per_trx": ":.2f",
        "PACKAGE_LITRE": False,
        "BRAND_CODE": False
    },
        labels={
        "total_sales": "Total Sales ($) ",
        "avg_price_per_litre": "Avg $/L ",
        "total_transactions": "Total Trx ",
        "avg_qty_per_trx": "Avg Units/Trx ",
        "avg_volume_per_trx": "Avg Volume/Trx (L) "
    },
    title="NZ Potting Mixes: Sales Volume by Brand & Package Size"
)

fig.update_layout(
    width=800,
    height=800,
    template="plotly_white",
    font=dict(family="Arial", size=14, color="#333"),
    title_font=dict(size=20),
    xaxis=dict(
        tickmode="array",
        tickvals=package_order,
        ticktext=[f"{x}L" for x in package_order],
        title="",
        tickfont=dict(size=13),
        titlefont=dict(size=16)
    ),
    yaxis=dict(
        title="",
        categoryorder="array",
        categoryarray=brand_order,
        tickfont=dict(size=13),
        titlefont=dict(size=16)
    ),
    coloraxis_colorbar=dict(
        title="Avg Volume/Trx (L)",
        titlefont=dict(size=14),
        tickfont=dict(size=12)
    )
)

# Optional: double bubble size (by halving sizeref)
max_sales = brand_pkg_summary["total_sales"].max()
desired_size_max = 30  # same unit as original size_max

fig.update_traces(
    marker=dict(
        sizemode="area",
        sizeref=0.25 * max_sales / desired_size_max**2,
        sizemin=5
    )
)

fig.show()

In [74]:
fig.write_html("potting_mix_volume_by_brand_NZ.html")

In [76]:
summary_by_package_full = (
    pm_nz.groupby('PACKAGE_LITRE')
    .agg(
        total_sales=('TOTAL_SALES', 'sum'),
        total_transactions=('TOTAL_TRX', 'sum'),
        avg_units_per_trx=('AVG_QTY_PER_TRX', 'mean'),
        avg_volume_per_trx=('ESTIMATED_LITRES_PER_TRX', 'mean'),
        avg_price_per_litre=('AVG_PRICE_PER_LITRE', 'mean'),
        num_items=('ITEM_NUMBER', 'nunique')
    )
    .reset_index()
    .sort_values(by='total_transactions', ascending=False)
)

summary_by_package_full

,PACKAGE_LITRE,total_sales,total_transactions,avg_units_per_trx,avg_volume_per_trx,avg_price_per_litre,num_items
2,40.0,7359092.75,427532,1.759384,70.375342,11.258775,6
1,30.0,3174556.28,164936,1.653407,49.602213,12.267848,16
0,25.0,622541.65,39110,1.435160,35.878994,11.166589,8


In [77]:
# Summarise volume trend by brand
volume_summary = (
    pm_nz.groupby(['BRAND_CODE', 'PACKAGE_LITRE'])
    .agg(
        avg_qty=('AVG_QTY_PER_TRX', 'mean'),
        total_trx=('TOTAL_TRX', 'sum'),
        avg_volume_per_trx=('ESTIMATED_LITRES_PER_TRX', 'mean'),
        unique_items=('ITEM_NUMBER', 'nunique')
    )
    .reset_index()
    .sort_values(['avg_volume_per_trx', 'BRAND_CODE'])
)
volume_summary

,BRAND_CODE,PACKAGE_LITRE,avg_qty,total_trx,avg_volume_per_trx,unique_items
1,Beut,30.0,1.000000,1,30.000000,1
15,SCOTTS,25.0,1.341664,8245,33.541606,2
13,OSMOCOTE,25.0,1.458240,28699,36.455995,4
3,DALTONS MEDIA,25.0,1.482495,2166,37.062379,2
16,SCOTTS,30.0,1.450726,2547,43.521790,1
12,NATURES WAY,30.0,1.541491,9847,46.244731,2
4,DALTONS MEDIA,30.0,1.554520,35418,46.635609,1
8,GARDEN BASICS,30.0,1.631493,15845,48.944778,1
6,DALTONS PREMIUM,30.0,1.640063,24948,49.201901,2
9,GARDEN TIME MEDIA,30.0,1.697000,63076,50.910002,4
